In [109]:
using LinearAlgebra
using SparseArrays
include("bArnoldi.jl")
include("arnoldi.jl")

arnoldi (generic function with 1 method)

In [110]:
n = 100
M = 30
L = 3
# We need this condition for now
M * L <= n

true

In [111]:
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
Γ = .7
# RHS
B = randn(n, L);

In [112]:
X_exact = zeros(n, L)
for l=1:L
    lhs = A # - D[l, l]*I
    b = B[:, l]
    X_exact[:, l] = lhs \ b
end

In [113]:
# Naive: Standard Arnoldi in a for-loop
X_σ_naive = zeros(n, L) 
for l=1:L
    lhs = A # - D[l, l]*I
    b = B[:, l]
    Q, H = arnoldi(lhs, b, M)
    for m in 1:M                         # Changed the following to be consistent with bArnoldi and gives smaller norms
        s = vcat(norm(b), zeros(m-1, 1)) #   size m+1 -> m
        z = H[1:m, 1:m] \ s              #   H[1:m+1, 1:m] -> H[1:m, 1:m]
        x = Q[:, 1:m] * z
        X_σ_naive[:, l] = x
    end
end

In [114]:
# Block Arnoldi
X_σ = zeros(n, L) 
Res = zeros(n, L)
W, H = bArnoldi(A, B, M);
@show norm(A*W[:,1:M*L] - W*H)

norm(A * W[:, 1:M * L] - W * H) = 1.6296131988461574e-13


1.6296131988461574e-13

In [115]:
f = qr(B)
for m in 1:M
    print("step $m \n")
    s = f.Q[:, 1:m*L]' * B   # s = Q' B
    z = H[1:L*m, 1:L*m] \ s  # z = H \ s
    x = W[:, 1:L*m] * z      # x = W_m z    
    X_σ = x
end


step 1 
step 2 
step 3 
step 4 
step 5 
step 6 
step 7 
step 8 
step 9 
step 10 
step 11 
step 12 
step 13 
step 14 
step 15 
step 16 
step 17 
step 18 
step 19 
step 20 
step 21 
step 22 
step 23 
step 24 
step 25 
step 26 
step 27 
step 28 
step 29 
step 30 


In [116]:
norm(X_σ - X_σ_naive), norm(X_exact - X_σ), norm(X_exact - X_σ_naive)

(1.6765580745931078e-9, 3.5324783623043346e-13, 1.676558546979575e-9)